# ECG Image Structure Analysis

## 1. Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import cv2
import warnings

warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize'] = (15, 8)

# Set paths
DATA_PATH = Path('/kaggle/input/physionet-ecg-image-digitization')
TRAIN_PATH = DATA_PATH / 'train'

# Load metadata
train_df = pd.read_csv(DATA_PATH / 'train.csv')
print(f"Loaded {len(train_df)} training samples")

## 2. Analyze Image Dimensions and Properties

In [ ]:
# Sample a few images and check their properties
sample_ids = train_df['id'].head(10).tolist()
image_properties = []

for sample_id in sample_ids:
    sample_dir = TRAIN_PATH / str(sample_id)
    # Check the original image (0001)
    img_path = sample_dir / f"{sample_id}-0001.png"

    if img_path.exists():
        img = Image.open(img_path)
        img_array = np.array(img)

        image_properties.append({
            'id': sample_id,
            'width': img.size[0],
            'height': img.size[1],
            'mode': img.mode,
            'channels': img_array.shape[2] if len(img_array.shape) == 3 else 1,
            'dtype': img_array.dtype,
            'min_val': img_array.min(),
            'max_val': img_array.max(),
            'mean_val': img_array.mean()
        })

props_df = pd.DataFrame(image_properties)
print("\nImage Properties Summary:")
print(props_df)

print("\nImage Dimensions Distribution:")
print(props_df[['width', 'height']].describe())

## 3. Visualize Image Structure with Annotations

In [ ]:
# Load a sample image and analyze its structure
sample_id = str(train_df['id'].iloc[0])
sample_dir = TRAIN_PATH / sample_id
img_path = sample_dir / f"{sample_id}-0001.png"

# Load with PIL and OpenCV
img_pil = Image.open(img_path)
img_cv = cv2.imread(str(img_path))
img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)
img_gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)

print(f"Sample ID: {sample_id}")
print(f"Image shape (RGB): {img_rgb.shape}")
print(f"Image shape (Gray): {img_gray.shape}")

# Display original and grayscale
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

axes[0].imshow(img_rgb)
axes[0].set_title('Original ECG Image (RGB)')
axes[0].axis('off')

axes[1].imshow(img_gray, cmap='gray')
axes[1].set_title('Grayscale ECG Image')
axes[1].axis('off')

plt.tight_layout()
plt.show()

## 4. Analyze Horizontal and Vertical Projections

In [ ]:
# Horizontal and vertical projections can help identify grid lines and lead regions
horizontal_projection = np.sum(img_gray, axis=1)
vertical_projection = np.sum(img_gray, axis=0)

fig, axes = plt.subplots(2, 2, figsize=(18, 10))

# Show image with projections
axes[0, 0].imshow(img_gray, cmap='gray')
axes[0, 0].set_title('ECG Image')
axes[0, 0].axis('off')

# Horizontal projection
axes[0, 1].plot(horizontal_projection, range(len(horizontal_projection)))
axes[0, 1].set_ylim(len(horizontal_projection), 0)
axes[0, 1].set_title('Horizontal Projection')
axes[0, 1].set_xlabel('Sum of pixel intensities')
axes[0, 1].set_ylabel('Row index')
axes[0, 1].grid(True, alpha=0.3)

# Vertical projection
axes[1, 0].plot(vertical_projection)
axes[1, 0].set_title('Vertical Projection')
axes[1, 0].set_xlabel('Column index')
axes[1, 0].set_ylabel('Sum of pixel intensities')
axes[1, 0].grid(True, alpha=0.3)

# Combined view
axes[1, 1].imshow(img_gray, cmap='gray')
axes[1, 1].set_title('ECG Image')
axes[1, 1].set_xlabel('Column index')
axes[1, 1].set_ylabel('Row index')

plt.tight_layout()
plt.show()

## 5. Detect Grid Lines

In [ ]:
# Try to detect horizontal and vertical grid lines using edge detection
edges = cv2.Canny(img_gray, 50, 150)

# Detect lines using Hough Transform
lines = cv2.HoughLinesP(edges, 1, np.pi/180, threshold=100, minLineLength=100, maxLineGap=10)

# Draw detected lines
img_with_lines = img_rgb.copy()
if lines is not None:
    print(f"Detected {len(lines)} line segments")

    # Separate horizontal and vertical lines
    horizontal_lines = []
    vertical_lines = []

    for line in lines:
        x1, y1, x2, y2 = line[0]
        angle = np.abs(np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi)

        if angle < 10 or angle > 170:  # Horizontal
            horizontal_lines.append(line)
            cv2.line(img_with_lines, (x1, y1), (x2, y2), (255, 0, 0), 1)
        elif 80 < angle < 100:  # Vertical
            vertical_lines.append(line)
            cv2.line(img_with_lines, (x1, y1), (x2, y2), (0, 255, 0), 1)

    print(f"Horizontal lines: {len(horizontal_lines)}")
    print(f"Vertical lines: {len(vertical_lines)}")

fig, axes = plt.subplots(1, 3, figsize=(20, 7))

axes[0].imshow(img_gray, cmap='gray')
axes[0].set_title('Original Image')
axes[0].axis('off')

axes[1].imshow(edges, cmap='gray')
axes[1].set_title('Edge Detection (Canny)')
axes[1].axis('off')

axes[2].imshow(img_with_lines)
axes[2].set_title(f'Detected Lines (H: blue, V: green)')
axes[2].axis('off')

plt.tight_layout()
plt.show()

## 6. Analyze Color Channels

In [ ]:
# Analyze RGB channels separately - ECG signals might be in specific channels
r_channel = img_rgb[:, :, 0]
g_channel = img_rgb[:, :, 1]
b_channel = img_rgb[:, :, 2]

fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Show each channel
axes[0, 0].imshow(r_channel, cmap='Reds')
axes[0, 0].set_title('Red Channel')
axes[0, 0].axis('off')

axes[0, 1].imshow(g_channel, cmap='Greens')
axes[0, 1].set_title('Green Channel')
axes[0, 1].axis('off')

axes[0, 2].imshow(b_channel, cmap='Blues')
axes[0, 2].set_title('Blue Channel')
axes[0, 2].axis('off')

# Histograms
axes[1, 0].hist(r_channel.flatten(), bins=50, color='red', alpha=0.7)
axes[1, 0].set_title('Red Channel Histogram')
axes[1, 0].set_xlabel('Pixel Value')
axes[1, 0].set_ylabel('Frequency')

axes[1, 1].hist(g_channel.flatten(), bins=50, color='green', alpha=0.7)
axes[1, 1].set_title('Green Channel Histogram')
axes[1, 1].set_xlabel('Pixel Value')
axes[1, 1].set_ylabel('Frequency')

axes[1, 2].hist(b_channel.flatten(), bins=50, color='blue', alpha=0.7)
axes[1, 2].set_title('Blue Channel Histogram')
axes[1, 2].set_xlabel('Pixel Value')
axes[1, 2].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

## 7. Extract and Visualize a Horizontal Slice

In [ ]:
# Extract a horizontal slice to see the signal pattern
slice_height = img_gray.shape[0] // 4  # Take a slice from 1/4 height
slice_data = img_gray[slice_height, :]

fig, axes = plt.subplots(2, 1, figsize=(18, 8))

# Show where the slice is taken from
axes[0].imshow(img_gray, cmap='gray')
axes[0].axhline(y=slice_height, color='r', linestyle='--', linewidth=2)
axes[0].set_title(f'ECG Image with Horizontal Slice at row {slice_height}')
axes[0].axis('off')

# Plot the slice
axes[1].plot(slice_data)
axes[1].set_title('Pixel Intensity Along Horizontal Slice')
axes[1].set_xlabel('Column Index (x)')
axes[1].set_ylabel('Pixel Intensity')
axes[1].grid(True, alpha=0.3)
axes[1].set_xlim(0, len(slice_data))

plt.tight_layout()
plt.show()

## 8. Compare Multiple Image Types

In [ ]:
# Compare different image types (original, scanned, photographed, damaged)
image_types_to_check = ['0001', '0003', '0004', '0005', '0010']
image_type_names = {
    '0001': 'Original',
    '0003': 'Color Scan',
    '0004': 'B&W Scan',
    '0005': 'Mobile Photo',
    '0010': 'Damaged'
}

fig, axes = plt.subplots(len(image_types_to_check), 2, figsize=(18, 4*len(image_types_to_check)))

for idx, img_type in enumerate(image_types_to_check):
    img_path = sample_dir / f"{sample_id}-{img_type}.png"

    if img_path.exists():
        img = cv2.imread(str(img_path))
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Show RGB
        axes[idx, 0].imshow(img_rgb)
        axes[idx, 0].set_title(f'{image_type_names.get(img_type, img_type)} - RGB')
        axes[idx, 0].axis('off')

        # Show grayscale histogram
        axes[idx, 1].hist(img_gray.flatten(), bins=50, alpha=0.7)
        axes[idx, 1].set_title(f'{image_type_names.get(img_type, img_type)} - Intensity Histogram')
        axes[idx, 1].set_xlabel('Pixel Value')
        axes[idx, 1].set_ylabel('Frequency')
        axes[idx, 1].grid(True, alpha=0.3)
    else:
        axes[idx, 0].text(0.5, 0.5, 'Image not found', ha='center', va='center')
        axes[idx, 0].axis('off')
        axes[idx, 1].text(0.5, 0.5, 'Image not found', ha='center', va='center')
        axes[idx, 1].axis('off')

plt.tight_layout()
plt.show()

## 9. Cropped Region Analysis

In [ ]:
# Take a closer look at a small region to see grid and signal details
# Crop a region from the middle of the image
crop_height = 400
crop_width = 800
start_y = (img_gray.shape[0] - crop_height) // 2
start_x = (img_gray.shape[1] - crop_width) // 2

cropped = img_rgb[start_y:start_y+crop_height, start_x:start_x+crop_width]
cropped_gray = img_gray[start_y:start_y+crop_height, start_x:start_x+crop_width]

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Show crop location
axes[0, 0].imshow(img_rgb)
axes[0, 0].add_patch(plt.Rectangle((start_x, start_y), crop_width, crop_height,
                                   fill=False, edgecolor='red', linewidth=2))
axes[0, 0].set_title('Full Image with Crop Region')
axes[0, 0].axis('off')

# Show cropped region
axes[0, 1].imshow(cropped)
axes[0, 1].set_title('Cropped Region (Color)')
axes[0, 1].axis('off')

# Show cropped grayscale
axes[1, 0].imshow(cropped_gray, cmap='gray')
axes[1, 0].set_title('Cropped Region (Grayscale)')
axes[1, 0].axis('off')

# Show inverted (dark signals on light background)
axes[1, 1].imshow(255 - cropped_gray, cmap='gray')
axes[1, 1].set_title('Cropped Region (Inverted)')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

## 10. Summary and Key Observations

In [ ]:
print("="*80)
print("IMAGE ANALYSIS SUMMARY")
print("="*80)
print("\n1. IMAGE PROPERTIES:")
print(f"   - Typical dimensions: {props_df['width'].mode().values[0]}x{props_df['height'].mode().values[0]}")
print(f"   - Color mode: {props_df['mode'].mode().values[0]}")
print(f"   - Pixel value range: {props_df['min_val'].min()}-{props_df['max_val'].max()}")

print("\n2. GRID STRUCTURE:")
if lines is not None:
    print(f"   - Horizontal grid lines detected: {len(horizontal_lines)}")
    print(f"   - Vertical grid lines detected: {len(vertical_lines)}")